## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/121-googleq-a-train-3fold-bert-base-unc-categoryv2/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/121-googleq-a-train-3fold-bert-base-unc-categoryv2/model_fold_1.h5', '/kaggle/input/121-googleq-a-train-3fold-bert-base-unc-categoryv2/model_fold_2.h5', '/kaggle/input/121-googleq-a-train-3fold-bert-base-unc-categoryv2/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512
N_CLASS_CAT = test['category'].nunique()

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output_seq = Dense(N_CLASS, activation="sigmoid", name="output_seq")(x)
    output_class = Dense(N_CLASS_CAT, activation="softmax", name="output_class")(pooled_output)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=[output_seq, output_class])
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test)[0][: ,:N_CLASS] / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.927318,0.648074,0.188183,0.549473,0.647343,0.552432,0.679916,0.609298,0.528219,...,0.900496,0.917316,0.571571,0.963811,0.968182,0.839239,0.098133,0.079360,0.804632,0.928977
1,46,0.890666,0.510079,0.009587,0.774297,0.794132,0.898136,0.522293,0.446752,0.065875,...,0.680737,0.940899,0.689133,0.962950,0.973353,0.855647,0.843525,0.128690,0.160521,0.862972
2,70,0.903183,0.688351,0.044261,0.815174,0.864589,0.870652,0.588024,0.515661,0.157142,...,0.857526,0.939980,0.626533,0.975558,0.971689,0.900728,0.149604,0.062612,0.811012,0.924597
3,132,0.852068,0.447859,0.008957,0.675882,0.793259,0.916768,0.532428,0.412811,0.171386,...,0.695490,0.936860,0.642420,0.967879,0.980709,0.873658,0.844188,0.191634,0.304431,0.882380
4,200,0.923497,0.557812,0.081830,0.788583,0.765091,0.706744,0.672137,0.608310,0.117711,...,0.770093,0.931089,0.633912,0.967759,0.960853,0.854578,0.134477,0.072941,0.598993,0.920742


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.884160,0.585380,0.039579,0.703697,0.797637,0.832838,0.579008,0.483830,0.224793,...,0.786594,0.929275,0.653971,0.963337,0.971376,0.860341,0.523948,0.133918,0.474017,0.898865
std,2812.670060,0.034457,0.100851,0.052361,0.090610,0.077999,0.114371,0.051252,0.077467,0.151413,...,0.075457,0.014524,0.036551,0.007225,0.008327,0.025474,0.296727,0.055416,0.240766,0.020689
min,39.000000,0.795538,0.412963,0.004766,0.324407,0.457242,0.301544,0.483895,0.377441,0.017160,...,0.595750,0.869625,0.532805,0.935359,0.936471,0.774565,0.011220,0.017963,0.062735,0.836225
25%,2572.000000,0.856061,0.496742,0.010337,0.652429,0.756927,0.796876,0.535858,0.426275,0.103137,...,0.723651,0.921119,0.631361,0.959306,0.965943,0.843265,0.225150,0.088565,0.269487,0.884710
50%,5093.000000,0.882224,0.575527,0.017827,0.702738,0.800528,0.875669,0.570938,0.454850,0.183913,...,0.793377,0.931320,0.655637,0.963836,0.972358,0.863159,0.612566,0.137658,0.432381,0.898905
75%,7482.000000,0.910232,0.660316,0.047617,0.759714,0.853434,0.909505,0.614616,0.517751,0.313128,...,0.853824,0.939697,0.677024,0.967997,0.977760,0.877824,0.800394,0.173856,0.663485,0.915644
max,9640.000000,0.966939,0.850033,0.338420,0.947374,0.948769,0.962665,0.723029,0.722829,0.723845,...,0.933316,0.964702,0.764418,0.979458,0.986838,0.923974,0.894892,0.291299,0.955046,0.947321
